<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/ChatGPT_Whisper_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio==3.20 pydub==0.25.1 gtts==2.3.1 openai==0.27.0

In [ ]:
import openai
from gtts import gTTS
import gradio as gr
from IPython.display import Audio
from pydub import AudioSegment

In [ ]:
openai.api_key = "***"

In [ ]:
class ChatGPTAPI:
    def __init__(self, system_setting):
        self.system = {"role": "system", "content": system_setting}
        self.input_list = [self.system]
        self.logs = []

    def input_message(self, input_text):
        self.input_list.append({"role": "user", "content": input_text})
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=self.input_list
        )
        self.logs.append(result)
        self.input_list.append(
            {"role": "assistant", "content": result.choices[0].message.content}
        )
        return self.input_list[-2]["content"], self.input_list[-1]["content"]


# 音声ファイルをWhisperのAPIに送信して文字起こしする関数を定義
def transcribe(audio):
    audio_file = open(audio, "rb")
    transcription = openai.Audio.transcribe("whisper-1", audio_file)
    text_user, text_chatgpt = api.input_message(transcription["text"])

    tts = gTTS(text=text_chatgpt, lang="ja", tld="co.jp", slow=False)  # gTTSオブジェクトを作成
    tts.save("hello_bing.mp3")  # 音声ファイルに保存
    sound = AudioSegment.from_mp3("hello_bing.mp3")
    sound_2x = sound.speedup(playback_speed=1.3)
    sound_2x.export("hello_bing.mp3", format="mp3")

    return "hello_bing.mp3"

In [ ]:
api = ChatGPTAPI(
    system_setting="あなたは常にポジティブな友達です。返答の文章は必ず30文字以内にしてください。では、会話を開始します。"
)

In [ ]:
audio_input = gr.Audio(source="microphone", type="filepath")
interface = gr.Interface(
    fn=transcribe, inputs=audio_input, outputs=[gr.Audio()]
)
interface.launch()